In [1]:
import threading
import queue
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
import torchvision
from torchvision import transforms
from datasets import load_dataset
import time
import random
# from submodlib import FacilityLocationFunction
import math
import os

/Users/yuktabagdi/miniconda3/envs/feature-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the CIFAR-10 dataset
cifar10 = load_dataset("cifar10")

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


Generating test split: 100%|██████████| 10000/10000 [00:00<00:00, 625548.70 examples/s]


In [3]:
def extract_features(img, resnet):
    """
    :param img: A CIFAR image
    :return: List of features
    """

    # Apply the transformation and convert the image to a tensor
    img_tensor = transform(img).unsqueeze(0)

    # Extract the features using the ResNet18 model
    with torch.no_grad():
        features = resnet(img_tensor)

    # Flatten the features and convert to a 1D numpy array
    features = features.squeeze().numpy()
    features = features.flatten()
    
    return features

In [4]:
# Create a threading.Event object
event = threading.Event()

def extract_features_resnet_threaded_cifar(cifar_dataset, num_threads=4 ):
    num_imgs = len(cifar_dataset)

    # Create a queue to store the image data
    img_queue = queue.Queue()

    # Create a list to store the extracted features
    features_list = []
    label_list = []
    index_list = []

    # Create a lock to protect the features list
    features_lock = threading.Lock()

    # Create a list of threads
    threads = []
    models = []

    # create multiple copies of the models for extracting features
    for i in range(num_threads):
        models.append(torch.hub.load('pytorch/vision:v0.11.3', 'resnet18', pretrained=True))

    # Start the threads
    for i in range(num_threads):
        thread = threading.Thread(target=extract_features_threaded_worker, args=(img_queue, index_list, features_list, label_list, features_lock, models[i], event))
        thread.start()
        threads.append(thread)

    # Enqueue all the image data
    for i, img in tqdm(enumerate(cifar_dataset)):
        img_queue.put((i, img["img"], img["label"]))

    # Signal to the threads that all of the images have been enqueued
    event.set()

    # Wait for all of the threads to finish
    for thread in threads:
        thread.join()

    # Create a DataFrame from the extracted features
    data = {'Index': index_list, 'Label': label_list, 'Features': features_list}
    df = pd.DataFrame(data)

    return df


In [5]:
def extract_features_threaded_worker(img_queue, index_list, features_list, label_list, features_lock, model, event):
    processed_images = 0
    while True:
        # Get an image path from the queue
        if img_queue.empty():
            # Wait for the main thread to signal that all of the images have been enqueued
            event.wait()

            # If the queue is still empty, break out of the loop
            if img_queue.empty():
                break

        index, img_path, label = img_queue.get()

        # Extract the features from the image
        img_features = extract_features(img_path, model)

        # Acquire the lock
        features_lock.acquire()

        # Add the extracted features to the list
        features_list.append(img_features)
        index_list.append(index)
        label_list.append(label)

        # Release the lock
        features_lock.release()
        
        # Increment the number of processed images
        processed_images+=1

        # If the thread has processed 1000 images, print the thread ID and the number of processed images
        if processed_images % 1000 == 0:
            print(f"Thread ID: {threading.current_thread().ident} | Processed images: {processed_images}")

        # If the queue is empty, break out of the loop
        if img_queue.empty():
            print(f"Thread ID: {threading.current_thread().ident} | Processed images: {processed_images}")
            break


In [6]:
def get_features_dataset(dataset, model):
    num_imgs = len(dataset)

    # Create a list to store the extracted features
    features_list = []
    label_list = []
    index_list = []

    # Enqueue all the image data
    for i, img in tqdm(enumerate(dataset)):
        features = extract_features(img["img"], model)
        features_list.append(features)
        label_list.append(img["label"])
        index_list.append(i)
        if i==5:
            break
        if i%100==0:
            print(time.strftime('%X'))

    # Create a DataFrame from the extracted features
    data = {'Index': index_list, 'Label': label_list, 'Features': features_list}
    df = pd.DataFrame(data)
    return df


In [7]:
def main():
    # Load the ResNet18 model
    resnet = torch.hub.load('pytorch/vision:v0.11.3', 'resnet18', pretrained=True)
    cifar_dataset = cifar10["train"]
    df = get_features_dataset(cifar_dataset, resnet)
    return df


In [8]:
# Load the ResNet18 model
resnet = torch.hub.load('pytorch/vision:v0.11.3', 'resnet18', pretrained=True)

df = extract_features_resnet_threaded_cifar(cifar10["train"])
# sort the dataframe by index
df = df.sort_values(by='Index')
# Convert the 'features' column to a NumPy array
# df['Features'] = np.array(df['Features'])

Using cache found in /Users/yuktabagdi/.cache/torch/hub/pytorch_vision_v0.11.3
/Users/yuktabagdi/miniconda3/envs/feature-env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yuktabagdi/miniconda3/envs/feature-env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Using cache found in /Users/yuktabagdi/.cache/torch/hub/pytorch_vision_v0.11.3
Using cache found in /Users/yuktabagdi/.cache/torch/hub/pytorch_vision_v0.11.3
Using cache found in /Users/yuktabagdi/.cache/torch/hub/pytorch_vision_v

Thread ID: 11171262464 | Processed images: 1000
Thread ID: 11137609728 | Processed images: 1000
Thread ID: 11154436096 | Processed images: 1000
Thread ID: 11120783360 | Processed images: 1000
Thread ID: 11171262464 | Processed images: 2000
Thread ID: 11154436096 | Processed images: 2000
Thread ID: 11120783360 | Processed images: 2000
Thread ID: 11137609728 | Processed images: 2000
Thread ID: 11171262464 | Processed images: 3000
Thread ID: 11154436096 | Processed images: 3000
Thread ID: 11137609728 | Processed images: 3000
Thread ID: 11120783360 | Processed images: 3000
Thread ID: 11171262464 | Processed images: 4000
Thread ID: 11154436096 | Processed images: 4000
Thread ID: 11120783360 | Processed images: 4000
Thread ID: 11137609728 | Processed images: 4000
Thread ID: 11171262464 | Processed images: 5000
Thread ID: 11154436096 | Processed images: 5000
Thread ID: 11120783360 | Processed images: 5000
Thread ID: 11137609728 | Processed images: 5000
Thread ID: 11154436096 | Processed image

In [12]:
print(type(df["Features"][0]))
import pickle

with open("dataframe.pkl", "wb") as f:
    pickle.dump(df, f)

<class 'numpy.ndarray'>


In [16]:
with open("dataframe.pkl", "rb") as f:
    loaded_df = pickle.load(f)

print(type(loaded_df["Features"][0]))

<class 'numpy.ndarray'>


In [ ]:
# Save the features DataFrame to a CSV file
filename = "features_cifar_check.csv"
df.to_csv(filename, index=False)

np_array = np.array(df['Features'])
flattened_array = np_array.tolist()
# Create a DataFrame from the flattened array
df = pd.DataFrame(flattened_array)
# Save the DataFrame to a CSV file
df.to_csv('np.csv', index=False, header=False)
